# Installers

In [ ]:
!pip install opencv-python
!pip install tqdm
!pip install matplotlib
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy

^C
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
Using cached idna-3.7-py3-none-any.whl (66 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.51.3 requires huggingface-hub<1.0,>=0.30.0, but you have huggingface-hub 0.24.7 which is incompatible.


Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ---------------------------------------- 4.1/4.1 MB 123.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
    --------------------------------------- 0.1/2.9 GB 307.3 MB/s eta 0:00:10
   - -------------------------------------- 0.1/2.9 GB 295.7 MB/s eta 0:00:10
   -- ------------------------------------- 0.2/2.9 GB 295.3 MB/s eta 0:00:10
   --- ------------------------------------ 0.2/2.9 GB 295.4 MB/s eta 0:00:10
   ---- ----------------------------------- 0.3/2.9 GB 289.1 MB/s eta 0:00:10
   ---- ----------------------------------- 0.4/2.9 GB 294.2 MB/s eta 0:00:09
   ----- ---------------------------------- 0.4/2.9 GB 288.7 MB/s eta 0:00:09
   ------ ----------------------

# Imports

In [ ]:
#utils
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import math
from PIL import Image
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
print(torch.cuda.is_available())
print(torch.__version__)
import random
import csv

#data
from google.colab import drive
import zipfile
import os



True
2.6.0+cu124


# Utils

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# Proper license plate decoding
def decode_plate(filename):
    try:
        code = filename.split('-')[4]
        indices = list(map(int, code.split('_')))
        plate = provinces[indices[0]] + alphabets[indices[1]] + ''.join([ads[i] for i in indices[2:]])
        return plate
    except:
        return None

In [ ]:
def ctc_collate_fn(batch):

    images, labels = zip(*batch)
    images = torch.stack(images, dim=0)  # [B, 3, 48, 144]
    targets = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels])
    target_lengths = torch.tensor([len(label) for label in labels], dtype=torch.long)
    input_lengths = torch.full(size=(len(labels),), fill_value=18, dtype=torch.long)

    return images, targets, input_lengths, target_lengths

In [ ]:
def ctc_decode(log_probs, idx2char, blank_idx):
    preds = log_probs.argmax(2).permute(1, 0)  # [B, T]
    decoded = []
    for pred in preds:
        seq = []
        prev = -1
        for p in pred:
            p = p.item()
            if p != blank_idx and p != prev:
                seq.append(p)
            prev = p
        decoded.append("".join([idx2char[i] for i in seq]))
    return decoded

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data

Dictionary Building

In [ ]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁",
             "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新",
             "警", "学", "O"]

alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S',
             'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']

ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T',
       'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']


base_chars = sorted(set(provinces + alphabets + ads))
base_chars.append("<BLANK>")

char2idx = {ch: i for i, ch in enumerate(base_chars)}
idx2char = {i: ch for ch, i in char2idx.items()}
vocab_size = len(base_chars)

BLANK_IDX = char2idx["<BLANK>"]

In [ ]:
zip_path = '/content/drive/MyDrive/Dataset_CV/RECOGNITION/CCPD2020_recogniton.zip'
extract_path = '/content/CCPD2020_recognition'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
train_img_dir = r"/content/CCPD2020_recognition/train"
val_img_dir = r"/content/CCPD2020_recognition/val"

In [ ]:
# Dataset Building for test
class CCPDTestDataset(Dataset):
    def __init__(self, img_dir, char2idx, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.char2idx = char2idx
        self.filenames = [
            f for f in os.listdir(img_dir)
            if f.endswith('.jpg') or f.endswith('.png')
        ]
        self.filenames = [f for f in self.filenames if decode_plate(f) is not None]

    def __len__(self):
        return len(self.filenames)

    def encode_plate(self, plate):
        return [self.char2idx[ch] for ch in plate]

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        plate = decode_plate(fname)
        label = torch.tensor(self.encode_plate(plate), dtype=torch.long)

        img_path = os.path.join(self.img_dir, fname)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return img, label

# Dataset Building for train e val
class CCPDLabelDataset(Dataset):
    def __init__(self, img_dir, char2idx, max_len=10, transform=None):
        self.img_dir = img_dir
        self.filenames = [
            f for f in os.listdir(img_dir)
            if f.endswith('.jpg') or f.endswith('.png')
        ]
        self.char2idx = char2idx
        self.max_len = max_len
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def encode_plate(self, plate):
        return [self.char2idx[ch] for ch in plate]

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        label_str = decode_plate(filename)
        if label_str is None:
            raise ValueError(f"Impossibile decodificare: {filename}")
        label = torch.tensor(self.encode_plate(label_str), dtype=torch.long)

        img_path = os.path.join(self.img_dir, filename)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label



In [ ]:
#87.24 accuracy con random erasing
train_transforms = transforms.Compose([
    transforms.Resize((48, 144)),

    transforms.RandomApply([
        transforms.RandomAffine(
            degrees=2, translate=(0.01, 0.01), scale=(0.97, 1.03), shear=2
        )
    ], p=0.5),

    transforms.RandomApply([
        transforms.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.1, hue=0.02
        ),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.8))
    ], p=0.4),

    transforms.RandomApply([
        transforms.RandomAdjustSharpness(sharpness_factor=1.5, p=0.5)
    ], p=0.5),

    transforms.ToTensor(),

    transforms.RandomErasing(
        p=0.2, scale=(0.01, 0.03), ratio=(0.3, 3.3), value='random'
    )
])

val_transforms = transforms.Compose([
    transforms.Resize((48, 144)),
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_dataset = CCPDLabelDataset(train_img_dir, char2idx, transform=train_transforms)
val_dataset = CCPDLabelDataset(val_img_dir, char2idx, transform=val_transforms)

g = torch.Generator()
g.manual_seed(42)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    collate_fn=ctc_collate_fn,
    pin_memory=True,
    generator=g,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=128,
    collate_fn=ctc_collate_fn,
    pin_memory=True
)

In [ ]:
test_img_dir = r"/content/CCPD2020_recognition/test"

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((48, 144)),
    transforms.ToTensor(),
])

test_dataset = CCPDTestDataset(test_img_dir, char2idx, transform=train_transforms)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=ctc_collate_fn,
    pin_memory=True
)

# Network

## IGFE

### Focus structure Module

Obiettivo: diminuire la dimensione spaziale dell'immagine senza perdita di informazione. \\

è il primo step di IGFE. \\

prende in input un tensore (3,48,144) (canale,altezza,larghezza) e deve fare poi: \\
1) slicing dell'immagine: si prendono pixel a intervalli alternati per ottenere 4 feature maps da 3 canali ciascuno -> (3, 24, 72) \\
2) concatenazione lungo il canale (torch.cat): si concatenano tutte queste mappe e si ottengono feature map di dimensione ->  (12, 24, 72)  \\
3) convoluzione (Conv2d): il paper non specifica i parametri usati. Yolov5 usa: Conv2d(in_channels=12, out_channels=32, kernel_size=3, stride=1, padding=1)


In [ ]:
class FocusStructure(nn.Module):
    def __init__(self, in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1):
        super(FocusStructure, self).__init__()
        self.conv = nn.Conv2d(in_channels * 4, out_channels, kernel_size, stride, padding)

    def forward(self, x):
        # slicing
        x1 = x[..., ::2, ::2]
        x2 = x[..., ::2, 1::2]
        x3 = x[..., 1::2, ::2]
        x4 = x[..., 1::2, 1::2]
        #concatenation
        x = torch.cat([x1, x2, x3, x4], dim=1)
        return self.conv(x)

### RESBLOCK

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CNNBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class RESBLOCK(nn.Module):
    def __init__(self, channels):
        super(RESBLOCK, self).__init__()
        self.cnn1 = CNNBlock(channels, channels)
        self.cnn2 = CNNBlock(channels, channels)

    def forward(self, x):
        out = self.cnn1(x)
        out = self.cnn2(out)
        return out + x  # residual connection

### ConvDownSampling

In [ ]:
class ConvDownSampling(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvDownSampling, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

### Feature extractor

In [ ]:
class IGFE(nn.Module):
    def __init__(self):
        super().__init__()
        self.focus = FocusStructure()            # [3, 48, 144] → [32, 24, 72]
        self.res1 = RESBLOCK(32)
        self.res2 = RESBLOCK(32)
        self.down1 = ConvDownSampling(32, 256)   # [12, 24, 72] → [256, 12, 36]
        self.res3 = RESBLOCK(256)
        self.res4 = RESBLOCK(256)
        self.down2 = ConvDownSampling(256, 512)  # [256, 12, 36] → [512, 6, 18]

    def forward(self, x):
        x = self.focus(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.down1(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.down2(x)
        return x  # Final Output: [B, 512, 6, 18]

## Encoder

Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, height, width):
        super(PositionalEncoding, self).__init__()
        self.height = height
        self.width = width

        if d_model % 4 != 0:
            raise ValueError("d_model must be divisible by 4 for 2D positional encoding")

        pe = torch.zeros(d_model, height, width)
        d_model_half = d_model // 2
        div_term = torch.exp(torch.arange(0, d_model_half, 2).float() * (-math.log(10000.0) / d_model_half))

        pos_w = torch.arange(0, width).unsqueeze(1)
        pos_h = torch.arange(0, height).unsqueeze(1)

        pe[0:d_model_half:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[1:d_model_half:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[d_model_half::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
        pe[d_model_half + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

        self.register_buffer('pe', pe.unsqueeze(0))  # [1, d_model, H, W]

    def forward(self, x):
        return x + self.pe[:, :, :x.size(2), :x.size(3)]

In [ ]:
class EncoderUnit(nn.Module):
    def __init__(self, d_model=512, d_mha=1024, n_heads=8, height=6, width=18):
        super(EncoderUnit, self).__init__()
        self.height = height
        self.width = width
        self.d_mha = d_mha

        self.conv1 = nn.Conv2d(d_model, d_mha, kernel_size=1)
        self.mha = nn.MultiheadAttention(d_mha, n_heads, batch_first=True)
        self.conv2 = nn.Conv2d(d_mha, d_model, kernel_size=1)
        self.norm = nn.LayerNorm(d_model)  # **LayerNorm **

    def forward(self, x):
        B, C, H, W = x.shape
        # Conv1 → [B, 1024, 6, 18]
        x1 = self.conv1(x)

        # Flatten for MHA → [B, 108, 1024]
        x1_seq = x1.view(B, self.d_mha, H * W).permute(0, 2, 1)

        # Multi-Head Attention
        attn_out, _ = self.mha(x1_seq, x1_seq, x1_seq)  # [B, 108, 1024]

        # Reshape a feature map and CNN BLOCK 2
        attn_out = attn_out.permute(0, 2, 1).view(B, self.d_mha, H, W)
        x2 = self.conv2(attn_out)  # [B, 512, 6, 18]

        # Residual + LayerNorm **(token-wise)**
        # Flatten of x e x2 → [B, 108, 512]
        x2_seq = x2.view(B, C, H * W).permute(0, 2, 1)
        x_res_seq = x.view(B, C, H * W).permute(0, 2, 1)

        norm_out = self.norm(x2_seq + x_res_seq)

        # return to the previous dimensions [B, 512, 6, 18]
        out = norm_out.permute(0, 2, 1).view(B, C, H, W)
        return out

class Encoder(nn.Module):
    def __init__(self, d_model=512, d_mha=1024, n_heads=8, num_units=3, height=6, width=18):
        super(Encoder, self).__init__()
        self.pe2d = PositionalEncoding(d_model, height, width)
        self.encoder_units = nn.ModuleList([
            EncoderUnit(d_model=d_model, d_mha=d_mha, n_heads=n_heads, height=height, width=width)
            for _ in range(num_units)
        ])

    def forward(self, x):
        # x: [B, 512, 6, 18]
        x = self.pe2d(x)
        for unit in self.encoder_units:
            x = unit(x)
        return x  # [B, 512, 6, 18]


## Decoder

In [ ]:
class PositionalEncoding1D(nn.Module):
    def __init__(self, d_model, max_len=18):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # [1, max_len, d_model]

    def forward(self, x):  # x: [B, T, D]
        if x.size(1) > self.pe.size(1):
            raise ValueError(f"Sequence length {x.size(1)} exceeds max_len={self.pe.size(1)} in PositionalEncoding1D.")
        return x + self.pe[:, :x.size(1)].to(x.device)

# === Masked Self-Attention ===
class MaskedMultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        B, T, _ = x.shape
        mask = torch.triu(torch.ones(T, T) * float('-inf'), diagonal=1).to(x.device)
        attn_output, _ = self.attn(x, x, x, attn_mask=mask)
        return self.norm(x + attn_output)


# === Cross-Attention ===
class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x, context):
        attn_output, _ = self.attn(x, context, context)
        return self.norm(x + attn_output)


# === Feed Forward Network ===
class FeedForwardNetwork(nn.Module):
    def __init__(self, embed_dim, dropout=0.1):
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim, embed_dim)
        )
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        return self.norm(x + self.ff(x))


# === Decoder Layer: masked self-attn → cross-attn → FFN ===
class DecoderLayer(nn.Module):
    def __init__(self, embed_dim=512, num_heads=8):
        super().__init__()
        self.self_attn = MaskedMultiHeadSelfAttention(embed_dim, num_heads)
        self.cross_attn = CrossAttention(embed_dim, num_heads)
        self.ffn = FeedForwardNetwork(embed_dim)

    def forward(self, x, encoder_out):
        x = self.self_attn(x)
        x = self.cross_attn(x, encoder_out)
        x = self.ffn(x)
        return x


# CNN BLOCK3 (2x1 kernel, stride=(3,1), padding=1)
class CNNBlock3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(512, 512, kernel_size=(2, 1), stride=(3, 1), padding=(1,0))
        self.norm = nn.BatchNorm2d(512)
        self.act = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.act(self.norm(self.conv(x)))  # [B, 512, 3, 18] → approx

# CNN BLOCK4 (1x1 kernel, stride=(3,1), padding=(0,1))
class CNNBlock4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(512, 512, kernel_size=1, stride=(3, 1), padding=(0, 0))
        self.norm = nn.BatchNorm2d(512)
        self.act = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.act(self.norm(self.conv(x)))  # [B, 512, 1, 18]



# === Final Parallel Decoder (CTC) ===
class ParallelTransformerDecoder(nn.Module):
    def __init__(self, num_classes, embed_dim=512, num_heads=8, num_layers=3, max_seq_len=18):
        super().__init__()
        self.block3 = CNNBlock3()
        self.block4 = CNNBlock4()
        self.pos_encoder = PositionalEncoding1D(embed_dim, max_len=max_seq_len)

        self.layers = nn.ModuleList([
            DecoderLayer(embed_dim, num_heads) for _ in range(num_layers)
        ])
        self.output_layer = nn.Linear(embed_dim, num_classes)

    def forward(self, encoder_out):  # encoder_out: [B, 512, 6, 18]
        x = self.block3(encoder_out)
        x = self.block4(x)
        x = x.squeeze(2).permute(0, 2, 1)
        x = self.pos_encoder(x)

        for i, layer in enumerate(self.layers):
            x = layer(x, x)
        logits = self.output_layer(x)
        return logits




# Training

In [ ]:
def train_model(model, train_loader, val_loader, vocab_size, device, blank_idx, epochs=20, checkpoint_path="best_model_green_cropped.pth"):
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CTCLoss(blank=blank_idx, zero_infinity=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True)


    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        total_loss, correct_seq, correct_char = 0, 0, 0
        total_char, total_samples = 0, 0
        print(f"\nEpoch {epoch + 1}/{epochs}")
        pbar = tqdm(train_loader, desc="Training")

        for images, targets, input_lengths, target_lengths in pbar:
            images, targets = images.to(device), targets.to(device)
            input_lengths, target_lengths = input_lengths.to(device), target_lengths.to(device)

            logits = model(images)
            log_probs = nn.functional.log_softmax(logits, dim=-1).permute(1, 0, 2)

            loss = criterion(log_probs, targets, input_lengths, target_lengths)

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            total_loss += loss.item()

            pred_ids = log_probs.permute(1, 0, 2).argmax(dim=-1)
            pred_cleaned = []
            for seq in pred_ids:
                cleaned = []
                prev = -1
                for p in seq:
                    p = p.item()
                    if p != prev and p != blank_idx:
                        cleaned.append(p)
                    prev = p
                pred_cleaned.append(cleaned)

            idx = 0
            for b in range(len(target_lengths)):
                gt_seq = targets[idx:idx + target_lengths[b]].tolist()
                idx += target_lengths[b]
                pred_seq = pred_cleaned[b]
                if pred_seq == gt_seq:
                    correct_seq += 1
                correct_char += sum(p == g for p, g in zip(pred_seq, gt_seq))
                total_char += len(gt_seq)
                total_samples += 1

        seq_acc = correct_seq / total_samples
        char_acc = correct_char / total_char if total_char > 0 else 0.0
        avg_train_loss = total_loss / len(train_loader)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Seq Acc: {seq_acc:.2%} | Char Acc: {char_acc:.2%}")

        # === Validation ===
        model.eval()
        with torch.no_grad():
            val_loss, val_seq, val_char, val_total_char, val_total_samples = 0, 0, 0, 0, 0

            for images, targets, input_lengths, target_lengths in val_loader:
                images, targets = images.to(device), targets.to(device)
                input_lengths, target_lengths = input_lengths.to(device), target_lengths.to(device)

                logits = model(images)
                log_probs = nn.functional.log_softmax(logits, dim=-1).permute(1, 0, 2)

                loss = criterion(log_probs, targets, input_lengths, target_lengths)
                val_loss += loss.item()

                pred_ids = log_probs.permute(1, 0, 2).argmax(dim=-1)
                pred_cleaned = []
                for seq in pred_ids:
                    cleaned = []
                    prev = -1
                    for p in seq:
                        p = p.item()
                        if p != prev and p != blank_idx:
                            cleaned.append(p)
                        prev = p
                    pred_cleaned.append(cleaned)

                idx = 0
                for b in range(len(target_lengths)):
                    gt_seq = targets[idx:idx + target_lengths[b]].tolist()
                    idx += target_lengths[b]
                    pred_seq = pred_cleaned[b]
                    if pred_seq == gt_seq:
                        val_seq += 1
                    val_char += sum(p == g for p, g in zip(pred_seq, gt_seq))
                    val_total_char += len(gt_seq)
                    val_total_samples += 1

            avg_val_loss = val_loss / len(val_loader)
            val_seq_acc = val_seq / val_total_samples
            val_char_acc = val_char / val_total_char if val_total_char > 0 else 0.0

            print(f"→ Val Loss: {avg_val_loss:.4f} | Seq Acc: {val_seq_acc:.2%} | Char Acc: {val_char_acc:.2%}")
            scheduler.step(avg_val_loss)

            print(f"→ LR: {scheduler.optimizer.param_groups[0]['lr']:.6f}")

            #  Saving Best Model
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save({
                    'model_state_dict': model.state_dict(),
                    'epoch': epoch + 1,
                    'val_loss': avg_val_loss,
                    'vocab_size': vocab_size
                }, checkpoint_path)
                print(f"The best model was saved at epoch {epoch+1} (val_loss: {avg_val_loss:.4f})")



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
set_seed(42)
# === Full License Plate Recognizer ===
class LicensePlateRecognizer(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.igfe = IGFE()  # Output: [B, 128, 6, 18]
        self.encoder = Encoder(d_model=512, d_mha=1024, n_heads=8, num_units=3, height=6, width=18)  # Output: [B, 512, 6, 18]

        self.decoder = ParallelTransformerDecoder(
            num_classes=vocab_size,
            embed_dim=512,
            num_heads=8,
            num_layers=3,
            max_seq_len=18
        )

    def forward(self, image):
        features = self.igfe(image)             # (B, 128, 6, 18)
        encoded = self.encoder(features)        # (B, 512, 6, 18)
        logits = self.decoder(encoded)          # (B, T, vocab_size)
        return logits

model = LicensePlateRecognizer(vocab_size=vocab_size).to(device)
checkpoint = torch.load("/content/drive/MyDrive/Computer_vision/pdlpr_weights_ccpd2019/pdlpr_ccpd2019.pth", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
print(f" Weighted model loaded from epoch {checkpoint['epoch']} (val_loss: {checkpoint['val_loss']:.4f})")
train_model(model, train_loader, val_loader, vocab_size, device, BLANK_IDX, epochs=20)


 Pesato modello caricato da epoca 109 (val_loss: 0.0024)

Epoch 1/20


Training:   0%|          | 0/46 [00:00<?, ?it/s]C:\Users\franc\AppData\Local\Temp\ipykernel_13404\2897291153.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels])
Training: 100%|██████████| 46/46 [02:39<00:00,  3.46s/it]


Epoch 1 | Train Loss: 0.2278 | Seq Acc: 86.03% | Char Acc: 94.93%
→ Val Loss: 0.0683 | Seq Acc: 89.31% | Char Acc: 97.43%
→ LR: 0.000100
 Miglior modello salvato a epoca 1 (val_loss: 0.0683)

Epoch 2/20


Training: 100%|██████████| 46/46 [01:11<00:00,  1.55s/it]


Epoch 2 | Train Loss: 0.0175 | Seq Acc: 97.28% | Char Acc: 99.41%
→ Val Loss: 0.0568 | Seq Acc: 92.41% | Char Acc: 98.25%
→ LR: 0.000100
 Miglior modello salvato a epoca 2 (val_loss: 0.0568)

Epoch 3/20


...

Epoch 18 | Train Loss: 0.0008 | Seq Acc: 99.77% | Char Acc: 99.96%
→ Val Loss: 0.0528 | Seq Acc: 93.71% | Char Acc: 98.75%
→ LR: 0.000013

Epoch 19/20


Training: 100%|██████████| 46/46 [00:31<00:00,  1.46it/s]


Epoch 19 | Train Loss: 0.0008 | Seq Acc: 99.81% | Char Acc: 99.97%
→ Val Loss: 0.0529 | Seq Acc: 93.51% | Char Acc: 98.73%
→ LR: 0.000013

Epoch 20/20


Training: 100%|██████████| 46/46 [00:34<00:00,  1.35it/s]


Epoch 20 | Train Loss: 0.0007 | Seq Acc: 99.93% | Char Acc: 99.98%
→ Val Loss: 0.0541 | Seq Acc: 93.61% | Char Acc: 98.70%
→ LR: 0.000013


# Evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
set_seed(42)
# === Full License Plate Recognizer ===
class LicensePlateRecognizer(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.igfe = IGFE()  # Output: [B, 128, 6, 18]
        self.encoder = Encoder(d_model=512, d_mha=1024, n_heads=8, num_units=3, height=6, width=18)  # Output: [B, 512, 6, 18]

        self.decoder = ParallelTransformerDecoder(
            num_classes=vocab_size,
            embed_dim=512,
            num_heads=8,
            num_layers=3,
            max_seq_len=18
        )

    def forward(self, image):
        features = self.igfe(image)             # (B, 128, 6, 18)
        encoded = self.encoder(features)        # (B, 512, 6, 18)
        logits = self.decoder(encoded)          # (B, T, vocab_size)
        return logits
# Calling the model
model = LicensePlateRecognizer(vocab_size=vocab_size).to(device)

# Loading the best Model
checkpoint = torch.load(r"/content/drive/MyDrive/Computer_vision/pdlpr_weights_ccpd2020/pdlpr_ccpd2020.pth", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

print(f" Loaded best model from epoch {checkpoint['epoch']} | Val Loss: {checkpoint['val_loss']:.4f}")

 Loaded best model from epoch 19 | Val Loss: 0.0411


In [ ]:
def test_model_ctc(model, test_loader, idx2char, blank_idx, device="cuda", save_csv=False):
    model.to(device)
    model.eval()

    correct_full = 0
    correct_char = 0
    total_char = 0
    total = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, targets, input_lengths, target_lengths in test_loader:
            images = images.to(device)
            targets = targets.to(device)

            log_probs = model(images)  # [B, T, C]
            log_probs = nn.functional.log_softmax(log_probs, dim=-1).permute(1, 0, 2)  # [T, B, C]

            preds = ctc_decode(log_probs, idx2char, blank_idx)

            idx = 0
            for b in range(images.size(0)):
                target_seq = targets[idx:idx + target_lengths[b]].tolist()
                target_text = "".join([idx2char[i] for i in target_seq])
                pred_text = preds[b]

                all_preds.append(pred_text)
                all_targets.append(target_text)

                if pred_text == target_text:
                    correct_full += 1
                correct_char += sum(p == t for p, t in zip(pred_text, target_text))
                total_char += len(target_text)
                total += 1
                idx += target_lengths[b]

                print(f"GT: {target_text} | Pred: {pred_text}")

    seq_acc = correct_full / total * 100
    char_acc = correct_char / total_char * 100
    print(f"\n Sequence Accuracy: {seq_acc:.2f}%")
    print(f" Character Accuracy: {char_acc:.2f}%")

    if save_csv:
        with open("ctc_predictions.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["GT", "Pred"])
            for gt, pred in zip(all_targets, all_preds):
                writer.writerow([gt, pred])
        print(" Saved Predictions in 'ctc_predictions.csv'")

In [ ]:
test_model_ctc(
    model=model,
    test_loader=test_loader,
    idx2char=idx2char,
    blank_idx=BLANK_IDX,
    device=device,
    save_csv=True
)

C:\Users\franc\AppData\Local\Temp\ipykernel_30344\756991085.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.cat([torch.tensor(label, dtype=torch.long) for label in labels])


GT: 皖AD66178 | Pred: 皖AD65170
GT: 皖AD32818 | Pred: 皖AD02818
GT: 皖AF01060 | Pred: 皖AF01060
GT: 皖AD67887 | Pred: 皖AD67887
GT: 皖ADB0129 | Pred: 皖AD00129
GT: 皖AD56789 | Pred: 皖AD56789
GT: 皖ADB3289 | Pred: 皖AD83289
GT: 皖AF08872 | Pred: 皖AF08872
GT: 皖AD18899 | Pred: 皖AD18899
GT: 皖AD11055 | Pred: 皖AD11055
GT: 皖AD49333 | Pred: 皖AD49333
GT: 皖AD57568 | Pred: 皖AD57568
GT: 皖AD62208 | Pred: 皖AD60708
GT: 皖AF07000 | Pred: 皖AF07000
GT: 皖AD50004 | Pred: 皖AD50004
GT: 皖AD56999 | Pred: 皖AD56999
GT: 皖AD49166 | Pred: 皖AD49166
GT: 皖AD18116 | Pred: 皖AD18116
GT: 皖AD53988 | Pred: 皖AD53988
GT: 皖AD49385 | Pred: 皖AD49385
GT: 皖AD99999 | Pred: 皖AD99999
GT: 皖AD48959 | Pred: 皖AD48959
GT: 皖AD01116 | Pred: 皖AD01116
GT: 皖ADB2856 | Pred: 皖AD62858
GT: 皖AD18577 | Pred: 皖AD18577
GT: 皖AD57566 | Pred: 皖AD57566
GT: 皖AD19618 | Pred: 皖AD19618
GT: 皖AD67887 | Pred: 皖AD67887
GT: 皖AD01668 | Pred: 皖AD01618
GT: 皖AF02822 | Pred: 皖AF02822
GT: 皖ADB2676 | Pred: 皖AD82676
GT: 皖AD11166 | Pred: 皖AD11166
GT: 皖AD47488 | Pred: 皖AD09108
GT: 皖AD229